In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import module, secret_keys
from model_list import models
import pandas as pd

hf_api_key             = secret_keys.HF_TOKEN                   #<insert your own huggingface token here>
openai_api_key         = secret_keys.OPENAI_API_KEY_TEAM        #<insert your own openai token here>

In [9]:
data = pd.read_csv('hidden_data/CT-Repo-With-Examples-Corrected-allgpteval.csv')

In [10]:
data.shape

(1693, 5)

In [11]:
data.head()

,NCTId,gpt4o_zs_gen_matches,gpt4o_ts_gen_matches,llama3_70b_it_zs_gen_matches,llama3_70b_it_ts_gen_matches
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00003901,"{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C..."
2,NCT00005879,"{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C..."
3,NCT00005908,"{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age Co...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age Co..."
4,NCT00006110,"{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, C...","{\n ""matched_features"": [\n [""Age, Continu...","{\n ""matched_features"": [\n [""Age, Continu..."


In [12]:
score_df = pd.DataFrame()
score_df['NCTId'] = data['NCTId']

In [ ]:
import json 

gzs_sum = {'precision':0, 'recall':0, 'f1':0}
gts_sum = {'precision':0, 'recall':0, 'f1':0}
lzs_sum = {'precision':0, 'recall':0, 'f1':0}
lts_sum = {'precision':0, 'recall':0, 'f1':0}

for index, row in data.iterrows():
    avoid_ids = ['NCT00000620', 'NCT01483560', 'NCT04280783'] #these were used as examples for 3-shot generation
    if row['NCTId'] in avoid_ids:
        continue

    #gzs
    gzs_dict = json.loads(row['gpt4o_zs_gen_matches'])
    gzs_matches, gzs_remaining_references, gzs_remaining_candidates = gzs_dict['matched_features'], gzs_dict['remaining_reference_features'], gzs_dict['remaining_candidate_features']
    gzs_score = module.match_to_score(gzs_matches, gzs_remaining_references, gzs_remaining_candidates)
    score_df.at[index, 'gpt4o_zs_gen_scores'] = json.dumps(gzs_score)
    gzs_sum['precision'] += gzs_score['precision']
    gzs_sum['recall'] += gzs_score['recall']
    gzs_sum['f1'] += gzs_score['f1']

    #gts
    gts_dict = json.loads(row['gpt4o_ts_gen_matches'])
    gts_matches, gts_remaining_references, gts_remaining_candidates = gts_dict['matched_features'], gts_dict['remaining_reference_features'], gts_dict['remaining_candidate_features']
    gts_score = module.match_to_score(gts_matches, gts_remaining_references, gts_remaining_candidates)
    score_df.at[index, 'gpt4o_ts_gen_scores'] = json.dumps(gts_score)
    gts_sum['precision'] += gts_score['precision']
    gts_sum['recall'] += gts_score['recall']
    gts_sum['f1'] += gts_score['f1']

    #lzs
    lzs_dict = json.loads(row['llama3_70b_it_zs_gen_matches'])
    lzs_matches, lzs_remaining_references, lzs_remaining_candidates = lzs_dict['matched_features'], lzs_dict['remaining_reference_features'], lzs_dict['remaining_candidate_features']
    lzs_score = module.match_to_score(lzs_matches, lzs_remaining_references, lzs_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_zs_gen_scores'] = json.dumps(lzs_score)
    lzs_sum['precision'] += lzs_score['precision']
    lzs_sum['recall'] += lzs_score['recall']
    lzs_sum['f1'] += lzs_score['f1']

    #lts 
    lts_dict = json.loads(row['llama3_70b_it_ts_gen_matches'])
    lts_matches, lts_remaining_references, lts_remaining_candidates = lts_dict['matched_features'], lts_dict['remaining_reference_features'], lts_dict['remaining_candidate_features']
    lts_score = module.match_to_score(lts_matches, lts_remaining_references, lts_remaining_candidates)
    score_df.at[index, 'llama3_70b_it_ts_gen_scores'] = json.dumps(lts_score)
    lts_sum['precision'] += lts_score['precision']
    lts_sum['recall'] += lts_score['recall']
    lts_sum['f1'] += lts_score['f1']

    #break



In [14]:
score_df.head()

,NCTId,gpt4o_zs_gen_scores,gpt4o_ts_gen_scores,llama3_70b_it_zs_gen_scores,llama3_70b_it_ts_gen_scores
0,NCT00000620,NaN,NaN,NaN,NaN
1,NCT00003901,"{""precision"": 0.2222222222222222, ""recall"": 0....","{""precision"": 0.5833333333333334, ""recall"": 0....","{""precision"": 0.45454545454545453, ""recall"": 0...","{""precision"": 0.6363636363636364, ""recall"": 0...."
2,NCT00005879,"{""precision"": 0.125, ""recall"": 0.3333333333333...","{""precision"": 0.2857142857142857, ""recall"": 0....","{""precision"": 0.25, ""recall"": 0.41666666666666...","{""precision"": 0.2608695652173913, ""recall"": 0...."
3,NCT00005908,"{""precision"": 0.125, ""recall"": 0.3333333333333...","{""precision"": 0.2857142857142857, ""recall"": 0....","{""precision"": 0.23529411764705882, ""recall"": 0...","{""precision"": 0.2857142857142857, ""recall"": 0...."
4,NCT00006110,"{""precision"": 0.15789473684210525, ""recall"": 0...","{""precision"": 0.3333333333333333, ""recall"": 0....","{""precision"": 0.25, ""recall"": 0.57142857142857...","{""precision"": 0.26666666666666666, ""recall"": 0..."


In [15]:
score_df.to_csv('hidden_data/CT-Repo-With-Examples-Corrected-gptmetric.csv', index=False)

In [17]:
size = 1690
print(f"GPT-4o Zero Shot Scores: Precision={gzs_sum['precision']/size} Recall={gzs_sum['recall']/size} F1={gzs_sum['f1']/size}")
print(f"GPT-4o Three Shot Scores: Precision={gts_sum['precision']/size} Recall={gts_sum['recall']/size} F1={gts_sum['f1']/size}")
print(f"Llama-3 70B Instruct Zero Shot Scores: Precision={lzs_sum['precision']/size} Recall={lzs_sum['recall']/size} F1={lzs_sum['f1']/size}")
print(f"Llama-3 70B Instruct Three Shot Scores: Precision={lts_sum['precision']/size} Recall={lts_sum['recall']/size} F1={lts_sum['f1']/size}")

GPT-4o Zero Shot Scores: Precision=0.27913091284825525 Recall=0.5171848460571707 F1=0.34162047312511207
GPT-4o Three Shot Scores: Precision=0.41312964820595643 Recall=0.5875408886489029 F1=0.4636974452359618
Llama-3 70B Instruct Zero Shot Scores: Precision=0.3327767802044299 Recall=0.5776381448763911 F1=0.4045934282278741
Llama-3 70B Instruct Three Shot Scores: Precision=0.41321419704536827 Recall=0.5711768620706456 F1=0.45581971519266734
